In [4]:
from abc import ABCMeta, abstractmethod
from typing import Tuple, Optional, List, Union, Dict
from pathlib import Path

import numpy as np
import pandas as pd

import sys
sys.path.append('/Users/usaito/workspace/obp-temp')
from estimator import InverseProbabilityWeighting, DirectMethod, DoublyRobust
from train_regression_model import train_eval_lgbm
from dataset import OBDWithContextSets
from obp.simulator import BanditSimulator
from obp.policy import Random, BernoulliTS
from logistic_bandit import LogisticTS, LogisticUCB
from obp.utils import estimate_confidence_interval_by_bootstrap

## Dataset

In [2]:
data_path = Path('.').resolve().parents[1] / 'obd'

In [3]:
obd = OBDWithContextSets(behavior_policy='rand', campaign='men', context_set='1', data_path=data_path)

In [4]:
train, val = obd.split_data()

In [5]:
train.keys()

dict_keys(['action', 'position', 'reward', 'pscore', 'X_policy', 'X_reg', 'X_user'])

## Train and Evaluate Regression Model for DM and DR

In [10]:
n_splits = 2
hyperparams = dict(max_iter=1000, learning_rate=0.01, min_samples_leaf=5, random_state=12345)
regression_model_dict, performance_dict = train_eval_lgbm(
    dataset=obd, n_splits=n_splits, hyperparams=hyperparams)

In [11]:
train['reward'].mean()

0.004857142857142857

In [12]:
regression_model_results = dict()
for metric, values in performance_dict.items():
    regression_model_results[metric] = estimate_confidence_interval_by_bootstrap(values)

In [13]:
pd.DataFrame(regression_model_results).T

,mean,lower,upper
auc,0.498990,0.404563,0.592834
rce,-0.020714,-0.042453,0.001352


## Section 5.1: Off-Policy Estimator Selection

In [17]:
sim = BanditSimulator()
policy = BernoulliTS(n_actions=obd.n_actions, len_list=3)

In [19]:
n_splits = 2
n_estimators = 5
np.random.seed(12345)

ope_results = {est: np.zeros(n_splits) for est in ['dm', 'ipw', 'dr']}
for random_state in np.arange(n_splits):
    ipw = InverseProbabilityWeighting()
    dm = DirectMethod(regression_model=regression_model_dict[random_state])
    dr = DoublyRobust(regression_model=regression_model_dict[random_state])
    estimator_dict = dict(dm=dm, ipw=ipw, dr=dr)

    train, test = obd.split_data(random_state=random_state)
    reward_test = test['reward']

    # bagging aggregation
    ope_results_temp = {est: np.zeros(n_estimators) for est in estimator_dict}
    for seed in np.arange(n_estimators):  # TODO: parallelization
        # run a bandit algorithm on logged bandit feedback
        boot_idx = np.random.choice(np.arange(obd.train_size), size=obd.train_size)
        train_boot = {key: arr[boot_idx] for key, arr in train.items()}
        simulation_log = sim.simulate(policy=policy, train=train_boot)
        # off-policy evaluation by using the result of the simulation
        for est_name, est in estimator_dict.items():
            ope_results_temp[est_name][seed] = est.estimate(
                simulation_log, train['X_user'], obd.X_action)
        # initialize policy parameters
        policy.initialize()

    # print results of each train-test split
    print(f'random_state={random_state}')
    ground_truth = np.mean(reward_test)
    for est_name, est in estimator_dict.items():
        estimated_policy_value = np.mean(ope_results_temp[est_name])
        relative_estimation_error_of_est = np.abs((estimated_policy_value - ground_truth) / ground_truth)
        ope_results[est_name][random_state] = relative_estimation_error_of_est
        print(f'{est_name.upper()}: {np.round(relative_estimation_error_of_est, 6)}')

 29%|██▉       | 2043/7000 [00:00<00:00, 20424.27it/s]random_state=0
DM: 0.21364
IPW: 0.271429
DR: 0.260185
100%|██████████| 7000/7000 [00:00<00:00, 22743.75it/s]
random_state=1
DM: 0.758096
IPW: 0.295238
DR: 0.340446


In [20]:
ope_results

{'dm': array([0.21364029, 0.75809628]),
 'ipw': array([0.27142857, 0.2952381 ]),
 'dr': array([0.2601853 , 0.34044627])}

In [23]:
# estimate confidence intervals by a nonparametric bootstrap method
ope_results_with_ci = {est: dict() for est in estimator_dict}
for est_name in estimator_dict.keys():
    ope_results_with_ci[est_name] =\
        estimate_confidence_interval_by_bootstrap(samples=ope_results[est_name])

In [24]:
pd.DataFrame(ope_results_with_ci).T

,mean,lower,upper
dm,0.486467,0.213640,0.758096
ipw,0.283469,0.271429,0.295238
dr,0.300147,0.260185,0.340446


In [26]:
simulation_log.head()

,action,reward,position,pscore,selected_action,indicator
0,33,0,2,0.029412,0,0
1,15,0,1,0.029412,26,0
2,21,0,3,0.029412,9,0
3,8,0,2,0.029412,7,0
4,4,0,2,0.029412,31,0


## Section 5.2 Counterfactual Policy Selection

In [28]:
sim = BanditSimulator()
policy = LogisticUCB(n_actions=obd.n_actions, len_list=3, dim=obd.dim_context, epsilon=0.1)

In [36]:
n_estimators = 15
random_state = 0
np.random.seed(12345)
ipw = InverseProbabilityWeighting()
dm = DirectMethod(regression_model=regression_model_dict[random_state])
dr = DoublyRobust(regression_model=regression_model_dict[random_state])
estimator_dict = dict(dm=dm, ipw=ipw, dr=dr)

train, test = obd.split_data(random_state=random_state)
reward_test = test['reward']
ground_truth = np.mean(reward_test)

# bagging aggregation
ope_results = {est: np.zeros(n_estimators) for est in estimator_dict}
relative_ope_results = {est: np.zeros(n_estimators) for est in estimator_dict}
for seed in np.arange(n_estimators):  # TODO: parallelization
    # run a bandit algorithm on logged bandit feedback
    boot_idx = np.random.choice(np.arange(obd.train_size), size=obd.train_size)
    train_boot = {key: arr[boot_idx] for key, arr in train.items()}
    simulation_log = sim.simulate(policy=policy, train=train_boot)
    # off-policy evaluation by using the result of the simulation
    for est_name, est in estimator_dict.items():
        estimated_policy_value =  est.estimate(simulation_log, train['X_user'], obd.X_action)
        ope_results[est_name][seed] = estimated_policy_value / ground_truth
    # initialize policy parameters
    policy.initialize()

# print results of each train-test split
print('=' * 25)
print(f'random_state={random_state}')
print('-----')
for est_name, est in estimator_dict.items():
    relative_estimated_policy_value = np.mean(ope_results[est_name])
    print(f'{est_name.upper()}: {np.round(relative_estimated_policy_value, 6)}')
print('=' * 25, '\n')

100%|██████████| 7000/7000 [00:03<00:00, 2036.24it/s]
random_state=0
-----
DM: 1.294161
IPW: 2.185714
DR: 2.102411



In [37]:
mean, upper, lower = estimate_confidence_interval_by_bootstrap(ope_results['dr'])

In [43]:
pd.DataFrame(dict(mean=mean, upper=upper, lower=lower), index=['ucb'])

,mean,upper,lower
ucb,2.105119,1.447319,2.727798


## Summarize Results